In [1]:
import sys
path1 = '/Users/steph/berkelab/DA_maze/DA/Modules/'
path2 = '/Users/steph/berkelab/DA_maze/Behavior/Modules/'
sys.path += [path1,path2]

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
from multi_rat_da import *
from tmat_ops import *
from hexLevelAnalyses import get_sigRats_fromMeanList
from photometryQuantifications import *
from scipy.stats import wilcoxon
from celluloid import Camera
from pdf2image import convert_from_path
%matplotlib qt

In [3]:
loadpath = "/Users/steph/berkelab/Data/" #location of dataset
#df = pd.read_csv(loadpath+"phot_decode_df_withHexStates.csv")
#df = pd.read_csv("/Volumes/Tim/Photometry/IM-1478/07162022/IM-1478_07162022_h_sampleframe.csv")

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

#df = reduce_mem_usage(df)

In [61]:
# pls show me the names of the dataframe columns bc I forget them thanks
df.columns

Index(['Unnamed: 0', 'x', 'y', 'green', 'port', 'rwd', 'block', 'pA', 'pB',
       'pC', 'frame', 'ref', 'vel', 'acc', 'tri', 'fromP', '470', 'beamA',
       'beamB', 'beamC', 'tot_tri', 'green_z_scored', 'fiberloc',
       'session_type', 'rat', 'date', 'nextprob', 'nextp', 'lenAC', 'lenBC',
       'lenAB', 'simple_rr', 'pchosen', 'dtop', 'nom_rwd_a', 'nom_rwd_b',
       'nom_rwd_c', 'hexlabels'],
      dtype='object')

In [124]:
# plot all x,y points (in a given block) to get a picture of the maze
bnum = 1
pad = 20
x = df.x[df.block==bnum]
y = df.y[df.block==bnum]
minx = np.nanmin(x)
miny = np.nanmin(y)
maxx = np.nanmax(x)
maxy = np.nanmax(y)
fig = plt.figure()
plt.plot(x, y, '.')
plt.xlim(minx-pad, maxx+pad)
plt.ylim(miny-pad, maxy+pad)
plt.axis('off')
plt.savefig('maze_background.png', bbox_inches='tight', transparent=True, pad_inches=0)

# same thing, but use a different color for each hex (rainbow order, yay)
fig = plt.figure()
hexcolor = iter(cm.rainbow(np.linspace(0, 1, 50)))    
for hex in range (1,50):
    c = next(hexcolor)
    plt.plot(x[df.hexlabels==hex], y[df.hexlabels==hex], '.', c=c, alpha=0.1)
plt.axis('off')
plt.savefig('maze_background_colored.png', bbox_inches='tight', transparent=True, pad_inches=0)

In [6]:
# test load image for plot background
img = plt.imread("maze_background.png")
fig, ax = plt.subplots()
ax.imshow(img, extent=[minx-pad, maxx+pad, miny-pad, maxy+pad])

In [7]:
# create plot of rat's xy (colored by hex) overlayed on the hex background

# these are empirical fudge factors so the hex background and the rat's xy position (kinda sorta) line up
# ideally we would just scale automatically but the rat's min/max xy aren't super reliable
xshift = -7
yshift = -35
imscale = 0

img = plt.imread("hex_background.png")
fig, ax = plt.subplots()
ax.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale])

hexcolor = iter(cm.rainbow(np.linspace(0, 1, 50)))    
x = df.x[df.block==1]
y = df.y[df.block==1]
for hex in range (1,50):
    c = next(hexcolor)
    plt.plot(x[df.hexlabels==hex], y[df.hexlabels==hex], '.', c=c, alpha=0.1)

In [8]:
# Animation of rat xy and predicted xy with background generated from all rat xy
img = plt.imread("maze_background_colored.png")
fig, ax = plt.subplots()

camera = Camera(fig)
for i in range(1000,2000):
    ax.imshow(img, extent=[minx-pad, maxx+pad, miny-pad, maxy+pad], alpha=0.3)
    plt.plot(df.x[i], df.y[i], color='grey', marker='o')
    plt.plot(df.x_pred[i], df.y_pred[i], 'yo')
    camera.snap()
animation = camera.animate(interval=10, blit=True)

In [5]:
# Animation of rat xy and predicted xy with hex background
img = plt.imread("hex_background.png")
fig, ax = plt.subplots()

camera = Camera(fig)
for i in range(1000,2000):
    ax.invert_yaxis()
    ax.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale], alpha=0.3)
    plt.plot(df.x[i], df.y[i], color='grey', marker='o')
    plt.plot(df.x_pred[i], df.y_pred[i], 'yo')
    ax.text(minx+xshift, miny+yshift, 'Rat in {}, pred in {}'.format(int(df.hexlabels[i]), df.pred_hexlabels[i]), verticalalignment='top', horizontalalignment='left', fontsize=15)
    camera.snap()
animation = camera.animate(interval=10, blit=True)

NameError: name 'xshift' is not defined

In [123]:
# Plot of rat xy colored by dopamine

# these are empirical fudge factors so the hex background and the rat's xy position (kinda sorta) line up
# ideally we would just scale automatically but the rat's min/max xy aren't super reliable
xshift = -7
yshift = -35
imscale = 0
trialnum = 6
blocknum = 2

xpred = df.x_pred[(df.block==blocknum)&(df.tri==trialnum)]
ypred = df.y_pred[(df.block==blocknum)&(df.tri==trialnum)]
x = df.x[(df.block==blocknum)&(df.tri==trialnum)]
y = df.y[(df.block==blocknum)&(df.tri==trialnum)]
dopamine = normalize(df.green_z_scored[(df.block==blocknum)&(df.tri==trialnum)])

img = plt.imread("hex_background.png")
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale])
ax1.scatter(x, y, c=dopamine, cmap='plasma', alpha=0.3)
ax2.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale])
ax2.scatter(xpred, ypred, c=dopamine, cmap='plasma', alpha=0.3)


NameError: name 'minx' is not defined

In [7]:
# add animation of this ^ trial
fig, ax = plt.subplots()
img = plt.imread("hex_background.png")

camera = Camera(fig)
for i in range(min(x.index), max(x.index)):
    #ax.invert_yaxis()
    ax.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale], alpha=0.3)
    plt.plot(x[i], y[i], color='grey', marker='o')
    plt.plot(xpred[i],ypred[i], 'yo')
    ax.text(minx+xshift, miny+yshift, 'Rat in {}, pred in {}'.format(int(df.hexlabels[i]), df.pred_hexlabels[i]), verticalalignment='top', horizontalalignment='left', fontsize=15)
    camera.snap()
animation = camera.animate(interval=10, blit=True)

Let's mess with transition matrices!

In [11]:
tmat = np.load('/Users/steph/berkelab/DA_maze/Data/Transition_Matrices/IM-1478/07252022/tmat_block_1.0.npy')

In [12]:
fig, ax = plt.subplots()
ax.imshow(tmat)


In [13]:
# when using Photrat to load things, make sure Tim's directory is mounted
remy = Photrat('IM-1478', '07252022')
remy.load_df()


Memory usage of dataframe is 387.17 MB
Memory usage after optimization is: 112.92 MB
Decreased by 70.8%


In [14]:
#tmat = remy.load_tmat()
tmo = TmatOperations(remy)
tmo.get_availstates()

TypeError: Photrat.__init__() missing 1 required positional argument: 'date'

In [ ]:
remy.create_hexdf(98.0, 1)

In [16]:
hexData = PhotRats(None)
# the directory containing your dataframe
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/data4sharing/"
datName = "hexLevelDf"
hexData.df = reduce_mem_usage(pd.read_csv(loadpath+datName+".csv",index_col=[0]))
hexData.directory_prefix = loadpath

Memory usage of dataframe is 52.45 MB
Memory usage after optimization is: 12.54 MB
Decreased by 76.1%


In [36]:
seshdict = "/Volumes/Tim/Photometry/10MfRatDataSet/sessionTable.csv"

photrats = PhotRats(seshdict)
photrats.directory_prefix = "/Volumes/Tim/Photometry/"
#photrats.load_tmats()

hexData = PhotRats(None)
# the directory containing your dataframe
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/data4sharing/"
datName = "hexLevelDf"
hexData.df = reduce_mem_usage(pd.read_csv(loadpath+datName+".csv",index_col=[0]))
hexData.directory_prefix = loadpath

In [46]:
seshdict = pd.read_csv("/Volumes/Tim/Photometry/10MfRatDataSet/sessionTable.csv")
photrats = PhotRats(seshdict)
photrats.directory_prefix = "/Volumes/Tim/Photometry/"
photrats.load_dfs()

could not load df for Unnamed: 0; 0


TypeError: 'NoneType' object does not support item assignment

All of my functions for things! 

In [6]:
def distance_squared(x1, y1, x2, y2):
    """ Calculates the squared Euclidean distance between two points """
    return (x2 - x1)**2 + (y2 - y1)**2

def find_turnaround(x_coords, y_coords):
    """ Find the index at which the rat on a dead end path turns around

    Parameters: x_coords (list); y_coords (list): 2 lists of x and y coordinates representing the path
    Returns: the index (int) at which the rat turns around, or None if no turnaround is detected
    """
    
    if len(x_coords) < 2 or len(y_coords) < 2 or len(x_coords) != len(y_coords):
        return None  # Invalid data
    
    max_distance_sq = 0
    turnaround_index = None
    
    for i in range(1, len(x_coords)):
        dist_sq = distance_squared(x_coords[0], y_coords[0], x_coords[i], y_coords[i])
        if dist_sq > max_distance_sq:
            max_distance_sq = dist_sq
            turnaround_index = i
    
    return turnaround_index

def animate_rat_and_pred(df, background_img):
    """ Animation of rat xy and predicted xy with given background image """
    
    fig, ax = plt.subplots()
    camera = Camera(fig)
    for i in df.index:
        #ax.invert_yaxis()
        #ax.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale], alpha=0.3)
        ax.imshow(background_img, alpha=0.3)
        plt.plot(df.x[i], df.y[i], color='grey', marker='o')
        plt.plot(df.x_pred[i], df.y_pred[i], 'yo')
        #ax.text(minx+xshift, miny+yshift, 'Rat in {}, pred in {}'.format(int(df.hexlabels[i]), df.pred_hexlabels[i]), verticalalignment='top', horizontalalignment='left', fontsize=15)
        #camera.snap()
    #animation = camera.animate(interval=10, blit=True)

1   HIToolbox                           0x00007ff811aef726 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 102
2   HIToolbox                           0x00007ff811aef2b8 _ZN15MenuBarInstance14EnableAutoShowEv + 52
3   HIToolbox                           0x00007ff811a5ecd7 _ZN15MenuBarInstance21UpdateAggregateUIModeE21MenuBarAnimationStylehhh + 1113
4   HIToolbox                           0x00007ff811aef173 _ZN15MenuBarInstance19SetFullScreenUIModeEjj + 175
5   AppKit                              0x00007ff80b5c04b7 -[NSApplication _setPresentationOptions:instance:flags:] + 1145
6   AppKit                              0x00007ff80b415165 -[NSApplication _updateFullScreenPresentationOptionsForInstance:] + 582
7   CoreFoundation                      0x00007ff80812f3f4 __CFNOTIFICATIONCENTER_IS_CALLING_OUT_TO_AN_OBSERVER__ + 137
8   CoreFoundation                      0x00007ff8081c943a ___CFXRegistrationPost_block_invoke + 88
9   CoreFoundation                      0x00007ff8081c9389 _CFXR

In [43]:
#plt.plot(dead_end_df.index)
plt.plot(dead_end_df.tri)

369628    185.375
369629    185.500
369630    185.625
369631    185.625
369632    185.750
           ...   
374192    179.125
374193    179.000
374194    178.750
374195    178.500
374196    178.250
Name: x, Length: 4569, dtype: float16

In [46]:
fig, ax = plt.subplots()
#ax.imshow(img, alpha=0.3)
ax.invert_yaxis()
ax.scatter(dead_end_df.x, dead_end_df.y, c='grey')
ax.scatter(dead_end_df.x_pred, dead_end_df.y_pred, c='yellow')
    #ax.text(minx+xshift, miny+yshift, 'Rat in {}, pred in {}'.format(int(df.hexlabels[i]), df.pred_hexlabels[i]), verticalalignment='top', horizontalalignment='left', fontsize=15)
    #camera.snap()
    #animation = camera.animate(interval=10, blit=True)

In [ ]:
# extract the stuff we care about once so plotting is way faster
x = df.x
y = df.y
minx = np.nanmin(x)
miny = np.nanmin(y)
maxx = np.nanmax(x)
maxy = np.nanmax(y)

xpred = df.x_pred[(df.block==blocknum)&(df.tri==trialnum)]
ypred = df.y_pred[(df.block==blocknum)&(df.tri==trialnum)]
x = df.x[(df.block==blocknum)&(df.tri==trialnum)]
y = df.y[(df.block==blocknum)&(df.tri==trialnum)]
dopamine = normalize(df.green_z_scored[(df.block==blocknum)&(df.tri==trialnum)])

img = plt.imread("hex_background.png")
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale])
ax1.scatter(x, y, c=dopamine, cmap='plasma', alpha=0.3)
ax2.imshow(img, extent=[minx+xshift-imscale, maxx+xshift+imscale, maxy+yshift+imscale, miny+yshift-imscale])
ax2.scatter(xpred, ypred, c=dopamine, cmap='plasma', alpha=0.3)